### TRABALHO 2

### Primeiro, vamos importar as bibliotecas necessárias.

In [593]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

### Inicializando janela

In [595]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 1600
largura = 1200
window = glfw.create_window(largura, altura, "PROJETO 3", None, None)
glfw.make_context_current(window)

In [596]:
especular = True # Define se será considerada apenas ambiente + difusa ou ambiente + difusa + especular (Phong)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec3.
* Definindo matrizes Model, View e Projection que acumulam transformações geométricas 3D e permitem navegação no cenário.
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base nas transformações Model, View e Projection.

In [599]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        attribute vec3 normals;
        
       
        varying vec2 out_texture;
        varying vec3 out_fragPos; //posicao do fragmento (i.e., posicao na superficie onde a iluminacao sera calculada)
        varying vec3 out_normal;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
            out_fragPos = vec3(  model * vec4(position, 1.0));
            out_normal = vec3( model *vec4(normals, 1.0));            
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [602]:
# Shader sem especular
if not especular:
    fragment_code = """
        uniform int isInternalObject; // Identifica se o objeto é interno (1) ou externo (0)
        uniform vec3 lightPos; // Posição da primeira luz
        uniform vec3 lightColor; // Cor da primeira luz
        uniform vec3 lightPos2; // Posição da segunda luz
        uniform vec3 lightColor2; // Cor da segunda luz
        uniform vec3 lightPos3; // Posição da terceira luz
        uniform vec3 lightColor3; // Cor da terceira luz
        uniform float ka1, kd1; // Coeficientes de reflexão ambiente e difusa da luz 1
        uniform float ka2, kd2; // Coeficientes de reflexão ambiente e difusa da luz 2
        uniform float ka3, kd3; // Coeficientes de reflexão ambiente e difusa da luz 3
        varying vec2 out_texture;
        varying vec3 out_normal;
        varying vec3 out_fragPos;
        uniform sampler2D samplerTexture;
        
        void main() {
            vec3 norm = normalize(out_normal); // Normaliza o vetor normal
            vec3 ambient = vec3(0.0);         // Inicializa o componente ambiente
            vec3 diffuse = vec3(0.0);         // Inicializa o componente difuso
        
            if (isInternalObject == 1) {
                // Cálculos para luzes internas (1 e 2)
                vec3 lightDir = normalize(lightPos - out_fragPos);
                float diff = max(dot(norm, lightDir), 0.0);
                diffuse += kd1 * diff * lightColor;
        
                vec3 lightDir2 = normalize(lightPos2 - out_fragPos);
                float diff2 = max(dot(norm, lightDir2), 0.0);
                diffuse += kd2 * diff2 * lightColor2;
        
                ambient += ka1 * lightColor + ka2 * lightColor2;
            } else if (isInternalObject == 0) {
                // Cálculos para luz externa (3)
                vec3 lightDir3 = normalize(lightPos3 - out_fragPos);
                float diff3 = max(dot(norm, lightDir3), 0.0);
                diffuse += kd3 * diff3 * lightColor3;
        
                ambient += ka3 * lightColor3;
            }
        
            vec4 texture = texture2D(samplerTexture, out_texture);
            vec4 result = vec4((ambient + diffuse), 1.0) * texture; // Modelo de iluminação
            gl_FragColor = result;
        }
    """

# Shader com especular
else:
    fragment_code = """
        uniform int isInternalObject; // Identifica se o objeto é interno (1) ou externo (0)
        uniform vec3 lightPos; // Posição da primeira luz
        uniform vec3 lightColor; // Cor da primeira luz
        uniform vec3 lightPos2; // Posição da segunda luz
        uniform vec3 lightColor2; // Cor da segunda luz
        uniform vec3 lightPos3; // Posição da terceira luz
        uniform vec3 lightColor3; // Cor da terceira luz
        uniform float ka1, kd1, ks1; // Coeficientes para luz 1
        uniform float ka2, kd2, ks2; // Coeficientes para luz 2
        uniform float ka3, kd3, ks3; // Coeficientes para luz 3
        uniform float ns; // Expoente especular
        uniform vec3 viewPos; // Posição da câmera
        varying vec2 out_texture;
        varying vec3 out_normal;
        varying vec3 out_fragPos;
        uniform sampler2D samplerTexture;
        
        void main() {
            vec3 norm = normalize(out_normal); // Normaliza o vetor normal
            vec3 viewDir = normalize(viewPos - out_fragPos); // Direção da câmera
            vec3 ambient = vec3(0.0);         // Inicializa o componente ambiente
            vec3 diffuse = vec3(0.0);         // Inicializa o componente difuso
            vec3 specular = vec3(0.0);        // Inicializa o componente especular
        
            if (isInternalObject == 1) {
                // Cálculos para luzes internas (1 e 2)
                vec3 lightDir = normalize(lightPos - out_fragPos);
                float diff = max(dot(norm, lightDir), 0.0);
                diffuse += kd1 * diff * lightColor;
        
                vec3 reflectDir = reflect(-lightDir, norm);
                float spec = pow(max(dot(viewDir, reflectDir), 0.0), ns);
                specular += ks1 * spec * lightColor;
        
                vec3 lightDir2 = normalize(lightPos2 - out_fragPos);
                float diff2 = max(dot(norm, lightDir2), 0.0);
                diffuse += kd2 * diff2 * lightColor2;

                vec3 reflectDir2 = reflect(-lightDir2, norm);
                float spec2 = pow(max(dot(viewDir, reflectDir2), 0.0), ns);
                specular += ks2 * spec2 * lightColor2;
        
                ambient += ka1 * lightColor + ka2 * lightColor2;
            } else if (isInternalObject == 0) {
                // Cálculos para luz externa (3)
                vec3 lightDir3 = normalize(lightPos3 - out_fragPos);
                float diff3 = max(dot(norm, lightDir3), 0.0);
                diffuse += kd3 * diff3 * lightColor3;
        
                vec3 reflectDir3 = reflect(-lightDir3, norm);
                float spec3 = pow(max(dot(viewDir, reflectDir3), 0.0), ns);
                specular += ks3 * spec3 * lightColor3;
        
                ambient += ka3 * lightColor3;
            }
        
            vec4 texture = texture2D(samplerTexture, out_texture);
            vec4 result = vec4((ambient + diffuse + specular), 1.0) * texture; // Modelo de iluminação
            gl_FragColor = result;
        }
       
    
    """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [604]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associando nosso código-fonte aos slots solicitados

In [606]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [608]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [610]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [612]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Linkagem do programa

In [614]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

# Definindo as cores das luzes
light_color1 = (0.0, 0.0, 1.0)  # Cor da primeira luz
light_color2 = (0.6, 1.0, 0.7)  # Cor da segunda luz 
light_color3 = (0.0, 1.0, 0.0)  # Cor da terceira luz

# Passando as cores das luzes para o shader
loc_light_color1 = glGetUniformLocation(program, "lightColor")
glUniform3f(loc_light_color1, *light_color1)

loc_light_color2 = glGetUniformLocation(program, "lightColor2")
glUniform3f(loc_light_color2, *light_color2)

loc_light_color3 = glGetUniformLocation(program, "lightColor3")
glUniform3f(loc_light_color3, *light_color3)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront.


Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file


Nos slides e vídeo-aula da Aula 11 - Parte 1, nós descrevemos o funcionamento desse formato.

In [617]:
#com trinagularizacao
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file and performs triangulation for faces with more than 3 vertices."""
    objects = {}
    vertices = []
    normals = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"):  # para cada linha do arquivo .obj
        if line.startswith('#'): continue  # ignora comentários
        values = line.split()  # quebra a linha por espaço
        if not values: continue

        # recuperando vértices
        if values[0] == 'v':
            vertices.append(values[1:4])

        # recuperando normais
        if values[0] == 'vn':
            normals.append(values[1:4])

        # recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        # recuperando material
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            face_normals = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                face_normals.append(int(w[2]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            # Triangulação: Se a face tiver mais de 3 vértices, divide em triângulos
            if len(face) > 3:
                for i in range(1, len(face) - 1):
                    # Triângulo formado pelos vértices: face[0], face[i], face[i+1]
                    faces.append((
                        [face[0], face[i], face[i + 1]],
                        [face_texture[0], face_texture[i], face_texture[i + 1]],
                        [face_normals[0], face_normals[i], face_normals[i + 1]],
                        material
                    ))
            else:
                faces.append((face, face_texture, face_normals, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces
    model['normals'] = normals

    return model

In [618]:
glHint(GL_LINE_SMOOTH_HINT, GL_DONT_CARE)
glEnable( GL_BLEND )
glBlendFunc( GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA )
glEnable(GL_LINE_SMOOTH)
glEnable(GL_TEXTURE_2D)
qtd_texturas = 37
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    print(img_textura,img.mode)
    img_width = img.size[0]
    img_height = img.size[1]
    #image_data = img.tobytes("raw", "RGB", 0, -1)
    image_data = img.convert("RGBA").tobytes("raw", "RGBA",0,-1)

    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, img_width, img_height, 0, GL_RGBA, GL_UNSIGNED_BYTE, image_data)

### A lista abaixo armazena todos os vertices carregados dos arquivos

In [620]:
faces_visited = []
vertices_list = []
textures_coord_list = []
normals_list = []

### Vamos carregar cada modelo e definir funções para desenhá-los

In [622]:
modelo = load_model_from_file('objetos/turtle/turtle.obj')
print('Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    if face[3] not in faces_visited:
        print(face[3],' vertice inicial =',len(vertices_list))
        faces_visited.append(face[3])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Vertice final:',len(vertices_list))
load_texture_from_file(0,'objetos/turtle/Turtle_body_BaseColor.png')
load_texture_from_file(1,'objetos/turtle/Turtle_eye_BaseColor.png')
load_texture_from_file(2,'objetos/turtle/Turtle_nails_BaseColor.png')
load_texture_from_file(3,'objetos/turtle/Turtle_glass_BaseColor.png')

Vertice inicial: 0
unghie2SG1  vertice inicial = 0
unghie2SG3  vertice inicial = 25872
unghie2SG2  vertice inicial = 30432
unghie2SG  vertice inicial = 34992
Vertice final: 51312
objetos/turtle/Turtle_body_BaseColor.png RGBA
objetos/turtle/Turtle_eye_BaseColor.png RGBA
objetos/turtle/Turtle_nails_BaseColor.png RGBA
objetos/turtle/Turtle_glass_BaseColor.png RGBA


In [623]:
modelo = load_model_from_file('objetos/rua/3Roads.obj')

print('Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    if face[3] not in faces_visited:
        print(face[3],' vertice inicial =',len(vertices_list))
        faces_visited.append(face[3])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Vertice final:',len(vertices_list))
# load_texture_from_file(5,'R1.png')
# load_texture_from_file(6,'R2.png')
load_texture_from_file(7,'objetos/rua/R3.png')

Vertice inicial: 51312
Road  vertice inicial = 51312
Road2  vertice inicial = 51348
Road1  vertice inicial = 51384
Vertice final: 51420
objetos/rua/R3.png RGB


In [624]:
modelo = load_model_from_file('objetos/Predios/Predios.obj')
print('Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    if face[3] not in faces_visited:
        print(face[3],' vertice inicial =',len(vertices_list))
        faces_visited.append(face[3])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Vertice final:',len(vertices_list))
load_texture_from_file(8,'objetos/Predios/BaseColor_2.jpg') 
load_texture_from_file(9,'objetos/Predios/BaseColor_1.jpg') 
load_texture_from_file(10,'objetos/Predios/BaseColor_3.jpg') 
load_texture_from_file(11,'objetos/Predios/BaseColor_4.jpg') 

Vertice inicial: 51420
Building_2  vertice inicial = 51420
Building_1  vertice inicial = 59268
Building_3  vertice inicial = 65058
Buidling_4  vertice inicial = 72222
Vertice final: 76386
objetos/Predios/BaseColor_2.jpg RGB
objetos/Predios/BaseColor_1.jpg RGB
objetos/Predios/BaseColor_3.jpg RGB
objetos/Predios/BaseColor_4.jpg RGB


In [625]:
modelo = load_model_from_file('objetos/calcada/calcada.obj')
print('Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Vertice final:', len(vertices_list))
load_texture_from_file(12,'objetos/calcada/calcada.jpg') #

Vertice inicial: 76386
Vertice final: 76392
objetos/calcada/calcada.jpg RGB


In [626]:
modelo = load_model_from_file('objetos/poste/poste.obj')
print('Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    if face[3] not in faces_visited:
        print(face[3],' vertice inicial =',len(vertices_list))
        faces_visited.append(face[3])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Vertice final:',len(vertices_list))
load_texture_from_file(13,'objetos/poste/tx_posteglass.png') 
load_texture_from_file(14,'objetos/poste/tx_postelamp.png') 
load_texture_from_file(15,'objetos/poste/tx_postebulb.png') 

Vertice inicial: 76392
glass  vertice inicial = 76392
Lamp  vertice inicial = 76728
bulb  vertice inicial = 89532
Vertice final: 93132
objetos/poste/tx_posteglass.png RGBA
objetos/poste/tx_postelamp.png RGB
objetos/poste/tx_postebulb.png RGBA


In [627]:
modelo = load_model_from_file('objetos/ambiente/skybox2.obj')
print('Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    if face[3] not in faces_visited:
        print(face[3],' vertice inicial =',len(vertices_list))
        faces_visited.append(face[3])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Vertice final:',len(vertices_list))
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(16,'objetos/ambiente/ceu2.png') 
load_texture_from_file(17,'objetos/ambiente/parede_chao.jpg') 

Vertice inicial: 93132
material5  vertice inicial = 93132
Vertice final: 93168
objetos/ambiente/ceu2.png RGBA
objetos/ambiente/parede_chao.jpg RGB


In [628]:
modelo = load_model_from_file('objetos/tapete/Rug.obj')
print('Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Vertice final:', len(vertices_list))
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(18,'objetos/tapete/rug.jpg') #

Vertice inicial: 93168
Vertice final: 2069040
objetos/tapete/rug.jpg RGB


In [629]:
modelo = load_model_from_file('objetos/rato/rato.obj')
print('Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Vertice final:', len(vertices_list))
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(19,'objetos/rato/rato2.png') #

Vertice inicial: 2069040
Vertice final: 2080146
objetos/rato/rato2.png RGBA


In [630]:
modelo = load_model_from_file('objetos/pizza/pizza.obj')
print('Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    if face[3] not in faces_visited:
        print(face[3],' vertice inicial =',len(vertices_list))
        faces_visited.append(face[3])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Vertice final:',len(vertices_list))

# Carregando texturas
load_texture_from_file(20, 'objetos/pizza/pizza1.jpg')
load_texture_from_file(21, 'objetos/pizza/pizza2.jpg')
load_texture_from_file(22, 'objetos/pizza/pizza3.png')
load_texture_from_file(23, 'objetos/pizza/pizza4.jpg')
load_texture_from_file(24, 'objetos/pizza/pizza5.png')

Vertice inicial: 2080146
Pizza_In  vertice inicial = 2080146
Pizza_Out  vertice inicial = 2215314
Pizza  vertice inicial = 2596242
Box  vertice inicial = 2608074
Logo  vertice inicial = 2609898
None  vertice inicial = 2610252
Vertice final: 2610690
objetos/pizza/pizza1.jpg RGB
objetos/pizza/pizza2.jpg RGB
objetos/pizza/pizza3.png RGBA
objetos/pizza/pizza4.jpg RGB
objetos/pizza/pizza5.png RGBA


In [631]:
modelo = load_model_from_file('objetos/escada/escada.obj')
print('Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )

print('Vertice final:',len(vertices_list))
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(25,'objetos/escada/escada.png')

Vertice inicial: 2610690
Vertice final: 2611002
objetos/escada/escada.png RGB


In [632]:
modelo = load_model_from_file('objetos/borboleta/borboleta.obj')
print('Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Vertice final:', len(vertices_list))
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(26,'objetos/borboleta/borboleta.bmp') #

Vertice inicial: 2611002
Vertice final: 2612430
objetos/borboleta/borboleta.bmp RGB


In [633]:
modelo = load_model_from_file('objetos/bueiro/bueiro.obj')
print('Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Vertice final:', len(vertices_list))
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(27,'objetos/bueiro/bueiro.png') #

Vertice inicial: 2612430
Vertice final: 5892132
objetos/bueiro/bueiro.png RGB


In [634]:
modelo = load_model_from_file('objetos/vagalume_/vagalume.obj')
print('Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Vertice final:', len(vertices_list))
# Carregando a textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(28, 'objetos/vagalume_/texturas/Back/DefaultMaterial_Base_Color.png')

Vertice inicial: 5892132
Vertice final: 8542980
objetos/vagalume_/texturas/Back/DefaultMaterial_Base_Color.png RGB


In [635]:
modelo = load_model_from_file('objetos/luz/luz.obj')
print('Processando modelo luz.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo luz.obj. Vertice final:',len(vertices_list))
load_texture_from_file(29,'objetos/luz/luz.png')

Processando modelo luz.obj. Vertice inicial: 8542980
Processando modelo luz.obj. Vertice final: 8543016
objetos/luz/luz.png RGB


In [636]:
modelo = load_model_from_file('objetos/fan/fan.obj')
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo cube.obj. Vertice final:',len(vertices_list))
load_texture_from_file(31,'objetos/fan/fan.png')

Processando modelo cube.obj. Vertice inicial: 8543016
Processando modelo cube.obj. Vertice final: 8570262
objetos/fan/fan.png RGB


In [637]:
modelo = load_model_from_file('objetos/2tv/tv.obj')
print('Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    if face[3] not in faces_visited:
        print(face[3],' vertice inicial =',len(vertices_list))
        faces_visited.append(face[3])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Vertice final:',len(vertices_list))
load_texture_from_file(4,'objetos/2tv/txt/tv/BaseColor.png')
load_texture_from_file(30,'objetos/2tv/txt/tv2/BaseColor.png')

Vertice inicial: 8570262
standardSurface1  vertice inicial = 8570262
Material.001  vertice inicial = 8596134
Vertice final: 8607234
objetos/2tv/txt/tv/BaseColor.png RGB
objetos/2tv/txt/tv2/BaseColor.png RGB


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.

Nós agora vamos requisitar dois slots.
* Um para enviar coordenadas dos vértices.
* Outros para enviar coordenadas de texturas.
* Um para enviar coordenadas de normals para iluminacao.

In [639]:
# Request a buffer slot from GPU
buffer = glGenBuffers(3)

###  Enviando coordenadas de vértices para a GPU

In [641]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

###  Enviando coordenadas de textura para a GPU

In [643]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)])
textures['position'] = textures_coord_list

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

###  Enviando dados de Iluminação a GPU

#### Dados de iluminação: vetores normais

In [646]:
normals = np.zeros(len(normals_list), [("position", np.float32, 3)]) # três coordenadas
normals['position'] = normals_list

# Upload coordenadas normals de cada vertice
glBindBuffer(GL_ARRAY_BUFFER, buffer[2])
glBufferData(GL_ARRAY_BUFFER, normals.nbytes, normals, GL_STATIC_DRAW)
stride = normals.strides[0]
offset = ctypes.c_void_p(0)
loc_normals_coord = glGetAttribLocation(program, "normals")
glEnableVertexAttribArray(loc_normals_coord)
glVertexAttribPointer(loc_normals_coord, 3, GL_FLOAT, False, stride, offset)

### Desenhando nossos modelos
* Cada modelo tem um Model para posicioná-los no mundo.
* É necessário saber qual a posição inicial e total de vértices de cada modelo
* É necessário indicar qual o ID da textura do modelo


In [648]:
ka1 = 0.1 # coeficiente de reflexao ambiente do modelo
kd1 = 0.5 # coeficiente de reflexao difusa do modelo
ks1 = 0.9 # coeficiente de reflexao especular do modelo
ka2 = 0.1 # coeficiente de reflexao ambiente do modelo
kd2 = 0.5 # coeficiente de reflexao difusa do modelo
ks2 = 0.9 # coeficiente de reflexao especular do modelo
ka3 = 0.1 # coeficiente de reflexao ambiente do modelo
kd3 = 0.5 # coeficiente de reflexao difusa do modelo
ks3 = 0.9 # coeficiente de reflexao especular do modelo

In [649]:
def desenha_parte(texture_id, start_vertex, num_vertices, rotacao_angulo, rotacao, escala, translacao):
    # Aplica a matriz model
    # Rotação
    angle = rotacao_angulo
    r_x, r_y, r_z = rotacao
    
    # Translação (posiciona a parte no espaço correto)
    t_x, t_y, t_z = translacao
    
    # Escala
    s_x, s_y, s_z = escala
    
    # Matriz de transformação model
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    # Define o ID da textura do modelo
    glBindTexture(GL_TEXTURE_2D, texture_id)
    
    # Desenha o modelo com os vértices e textura apropriados
    glDrawArrays(GL_TRIANGLES, start_vertex, num_vertices)  # renderizando a parte específica

In [650]:
def desenha_turtle(pos, esc, angrot):
    # Configurações comuns para a tartaruga
    rotacao = (0.0, 1.0, 0.0)   # Rotação ao redor do eixo Y
    translacao = pos
    escala = (esc, esc, esc)
    
    # Desenha cada parte da tartaruga usando a função desenha_parte
    desenha_parte(
        texture_id=0,
        start_vertex=0,
        num_vertices=25872,          # Número de vértices para a primeira parte
        rotacao_angulo=210.0 + angrot,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

    desenha_parte(
        texture_id=1,
        start_vertex=25872,
        num_vertices=30432 - 25872,  # Número de vértices para a segunda parte
        rotacao_angulo=210.0 + angrot,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

    desenha_parte(
        texture_id=2,
        start_vertex=30432,
        num_vertices=34992 - 30432,  # Número de vértices para a terceira parte
        rotacao_angulo=210.0 + angrot,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

    desenha_parte(
        texture_id=3,
        start_vertex=34992,
        num_vertices=51312 - 34992,  # Número de vértices para a quarta parte
        rotacao_angulo=210.0 + angrot,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

In [651]:
def desenha_rua():
    # Configurações de transformação para a rua
    rotacao = (0.0, 1.0, 0.0)  # Rotação ao redor do eixo Y
    translacao = (+3.5 , +0.0  , -5.20)
    escala = (0.1 * 5, 0.1 * 1, 0.1 * 18)  # Escala ajustada conforme especificado

    # Desenha a rua usando a função desenha_parte
    desenha_parte(
        texture_id=7,
        start_vertex=51384,
        num_vertices=51420 - 51384,  # Número de vértices para a rua
        rotacao_angulo=210.0,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

In [652]:
def desenha_predios12():
    # Configurações de transformação para os prédios
    rotacao = (0.0, 1.0, 0.0)  # Rotação ao redor do eixo Y
    translacao = (+5.5 , 0.0 , -2.30)
    escala = (0.15, 0.15, 0.15)  # Escala ajustada conforme especificado

    # Primeiro conjunto de vértices e textura
    desenha_parte(
        texture_id=8,
        start_vertex=51420,
        num_vertices=59268 - 51420,  # Número de vértices do primeiro conjunto
        rotacao_angulo=29.0,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

    # Segundo conjunto de vértices e textura
    desenha_parte(
        texture_id=9,
        start_vertex=59268,
        num_vertices=65058 - 59268,  # Número de vértices do segundo conjunto
        rotacao_angulo=29.0,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

In [653]:
def desenha_predios34():
    # Configurações de transformação para os prédios 34
    rotacao = (0.0, 1.0, 0.0)  # Rotação ao redor do eixo Y
    translacao = (0.0, 0.0, 0.0)
    escala = (0.15, 0.15, 0.15)  # Escala especificada

    # Desenha a primeira parte dos prédios usando a função desenha_parte
    desenha_parte(
        texture_id=10,
        start_vertex=65058,
        num_vertices=72222 - 65058,  # Número de vértices para a primeira parte dos prédios
        rotacao_angulo=210.0,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

    # Desenha a segunda parte dos prédios usando a função desenha_parte
    desenha_parte(
        texture_id=11,
        start_vertex=72222,
        num_vertices=76386 - 72222,  # Número de vértices para a segunda parte dos prédios
        rotacao_angulo=210.0,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

In [654]:
def desenha_calcada():
    # Configurações de transformação para a calçada
    rotacao = (0.0, 1.0, 0.0)  # Rotação ao redor do eixo Y
    translacao = (+1.3 , +0.0  , -4.0)
    escala = (0.03, 0.03, 0.03)  # Escala especificada

    # Desenha a calçada usando a função desenha_parte
    desenha_parte(
        texture_id=12,
        start_vertex=76386,
        num_vertices=76392 - 76386,  # Número de vértices para a calçada
        rotacao_angulo=210.0,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

In [655]:
def desenha_poste(pos, esc):
    # Configurações de transformação para o poste
    rotacao = (0.0, 1.0, 0.0)  # Rotação ao redor do eixo Y
    translacao = pos
    escala = (0.1 * esc, 0.1 * esc, 0.1 * esc)  # Escala ajustada conforme especificado

    # Desenha cada parte do poste usando a função desenha_parte

    desenha_parte(
        texture_id=13,
        start_vertex=76392,
        num_vertices=76728 - 76392,  # Número de vértices para a primeira parte
        rotacao_angulo=210.0,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

    desenha_parte(
        texture_id=14,
        start_vertex=76728,
        num_vertices=89532 - 76728,  # Número de vértices para a segunda parte
        rotacao_angulo=210.0,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

    desenha_parte(
        texture_id=15,
        start_vertex=89532,
        num_vertices=93132 - 89532,  # Número de vértices para a terceira parte
        rotacao_angulo=210.0,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

In [656]:
def desenha_ceu():
    # Configurações de transformação para o céu
    rotacao = (0.0, 1.0, 0.0)  # Rotação ao redor do eixo Y
    translacao = (+1.2, +0.0, -4.0)
    escala = (1.1, 1.9, 1.1)  # Escala ajustada conforme especificado

    # Loop para desenhar cada face do céu
    for i in range(6):
        desenha_parte(
            texture_id=16,
            start_vertex=93132 + 6 * i,
            num_vertices=6,  # Cada face possui 6 vértices
            rotacao_angulo=210.0,
            rotacao=rotacao,
            escala=escala,
            translacao=translacao
        )

In [657]:
def desenha_casa():
    # Configurações de transformação para a casa
    rotacao = (0.0, 0.0, 1.0)  # Rotação ao redor do eixo Z
    translacao = (+1.4, -2.41, -4.3)
    escala = (1 * 0.6, 1 * 0.6, 1 * 0.6)  # Escala ajustada conforme especificado

    # Desenha (texture_id 17)
    for i in range(0, 6):
        desenha_parte(
            texture_id=17,
            start_vertex=93132 + 6 * i,
            num_vertices=6,  # Número de vértices para cada parte da casa
            rotacao_angulo=210.0 + 59.8,
            rotacao=rotacao,
            escala=escala,
            translacao=translacao
        )

In [658]:
def desenha_tapete():
    # Configurações de transformação para o tapete
    rotacao = (0.0, 0.0, 1.0)  # Rotação ao redor do eixo Z
    translacao = (+1.4, -4.8, -4.1)
    escala = (1 * 0.9, 1 * 0.9, 1 * 0.9)  # Escala ajustada conforme especificado

    # Desenha o tapete (texture_id 18)
    desenha_parte(
        texture_id=18,
        start_vertex=93168,
        num_vertices=2069040 - 93168,  # Número de vértices para o tapete
        rotacao_angulo=210.0 + 150.2,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

In [659]:
def desenha_rato():
    # Configurações de transformação para o rato
    rotacao = (0.0, 0.0, 1.0)  # Rotação ao redor do eixo Z
    translacao = (+3.1, -4.6, -2.5)
    escala = (1 * 0.5, 1 * 0.5, 1 * 0.5)  # Escala ajustada conforme especificado

    # Desenha o rato (texture_id 19)
    desenha_parte(
        texture_id=19,
        start_vertex=2069040,
        num_vertices=2080146 - 2069040,  # Número de vértices para o rato
        rotacao_angulo=210.0 + 144.4,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

In [660]:
def desenha_pizza():
    rotacao = (0.0, 1.0, 1.0)  # Rotação ao redor do eixo Y e Z
    translacao = (-0.30000001192092896, -3.1000008583068848, -2.799999952316284)
    escala = (1 * 0.2, 1 * 0.2, 1 * 0.2)  # Escala ajustada conforme especificado

    # Desenha a primeira parte da pizza (texture_id 20)
    desenha_parte(
        texture_id=20,
        start_vertex=2080146,
        num_vertices=2215314 - 2080146,  # Número de vértices para a primeira parte
        rotacao_angulo=210.0 + 510.8,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

    # Desenha a segunda parte da pizza (texture_id 21)
    desenha_parte(
        texture_id=21,
        start_vertex=2215314,
        num_vertices=2596242 - 2215314,  # Número de vértices para a segunda parte
        rotacao_angulo=210.0 + 510.8,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

    # Desenha a terceira parte da pizza (texture_id 22)
    desenha_parte(
        texture_id=22,
        start_vertex=2596242,
        num_vertices=2608074 - 2596242,  # Número de vértices para a terceira parte
        rotacao_angulo=210.0 + 510.8,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

In [661]:
def desenha_tampa(pos, angrot):
    # Configurações de transformação para a tampa
    rotacao = (1.0, 0.0, 0.0)  # Rotação ao redor do eixo X
    translacao = (pos[0], pos[1], pos[2])
    escala = (0.2, 0.2, 0.2)  # Escala ajustada conforme especificado

    # Desenha a tampa utilizando a função desenha_parte para a tampa
    desenha_parte(
        texture_id=23,
        start_vertex=2608074,
        num_vertices=2609898 - 2608074,  # Número de vértices para a tampa
        rotacao_angulo=210.0 + angrot,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

    # Desenha o logo utilizando a função desenha_parte para o logo
    desenha_parte(
        texture_id=24,
        start_vertex=2609898,
        num_vertices=2610252 - 2609898,  # Número de vértices para o logo
        rotacao_angulo=210.0 + angrot,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

In [662]:
def desenha_escada(pos, esc, angrot):
    # Configurações de transformação para a escada
    rotacao = (1.0, 0.0, 0.0)  # Rotação ao redor do eixo X
    translacao = pos
    escala = (1 * esc, 1 * esc, 1 * esc)  # Escala ajustada conforme especificado

    # Desenha a escada utilizando a função desenha_parte
    desenha_parte(
        texture_id=25,
        start_vertex=2610690,
        num_vertices=2611002 - 2610690,  # Número de vértices para a tampa da escada
        rotacao_angulo=210.0 + angrot,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

In [663]:
def desenha_borboleta(pos, esc, angrot):
    # Define os parâmetros de rotação, escala e translação
    rotacao = (1.0, 0.0, 0.0)  # Eixo de rotação (x, y, z)
    translacao = pos
    escala = (esc, esc, esc)  # Escala uniforme

    desenha_parte(
            texture_id=26,               # ID da textura
            start_vertex=2611002,        # Vértice inicial
            num_vertices=2612430 - 2611002, # Número de vértices
            rotacao_angulo=210.0 + angrot, # Ângulo de rotação
            rotacao=rotacao,             # Eixo de rotação
            escala=escala,               # Escala aplicada
            translacao=translacao        # Translação aplicada
        )

In [664]:
def desenha_bueiro():
    # Configurações de transformação para o bueiro
    rotacao = (0.0, 1.0, 0.0)  # Rotação ao redor do eixo Y
    translacao = (+0.9, 0.02, -1.95)
    escala = (0.1, 0.1 , 0.1)  # Escala ajustada conforme especificado

    # Desenha o bueiro utilizando a função desenha_parte
    desenha_parte(
        texture_id=27,
        start_vertex=2612430,
        num_vertices=5892132 - 2612430,  # Número de vértices para o bueiro
        rotacao_angulo=210.0,
        rotacao=rotacao,
        escala=escala,
        translacao=translacao
    )

In [665]:
def desenha_vagalume():
    global ka3, kd3, ks3, vagalume_luz_position

    # Usa a posição compartilhada
    t_x, t_y, t_z = vagalume_luz_position.x, vagalume_luz_position.y, vagalume_luz_position.z

    # Matriz model
    angle = 0.0
    r_x, r_y, r_z = 0.0, 1.0, 0.0
    s_x, s_y, s_z = 0.1 * 0.003, 0.1 * 0.003, 0.1 * 0.003
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
    
    # Configura e desenha a tartaruga
    loc_ka = glGetUniformLocation(program, "ka3")
    glUniform1f(loc_ka, ka3)
    loc_kd = glGetUniformLocation(program, "kd3")
    glUniform1f(loc_kd, kd3)

    if especular:
        ns = ns_inc
        loc_ks = glGetUniformLocation(program, "ks3")
        glUniform1f(loc_ks, ks3)
        loc_ns = glGetUniformLocation(program, "ns")
        glUniform1f(loc_ns, ns)
    
    glBindTexture(GL_TEXTURE_2D, 28)
    glDrawArrays(GL_TRIANGLES, 5892132, 8542980-5892132)

In [666]:
def desenha_luz(esc, t_x, t_y, t_z, light_id):
    global ka1, kd1, ks1, ka2, kd2, ks2, ka3, kd3, ks3, vagalume_luz_position

    # Selecionar coeficientes de reflexão baseados no ID da luz
    if light_id == 1:
        ka, kd, ks = ka1, kd1, ks1  # Valores para a luz 1
    elif light_id == 2:
        ka, kd, ks = ka2, kd2, ks2  # Valores para a luz 2
    elif light_id == 3:
        ka, kd, ks = ka3, kd3, ks3  # Valores para a luz 3

    # Enviar coeficientes de reflexão para o shader
    loc_ka = glGetUniformLocation(program, "ka")
    glUniform1f(loc_ka, ka)  # Envia ka para a GPU
    
    loc_kd = glGetUniformLocation(program, "kd")
    glUniform1f(loc_kd, kd)  # Envia kd para a GPU
    
    if especular:
        ns = 1000.0  # Valor alto para reflexão especular
        loc_ks = glGetUniformLocation(program, "ks")
        glUniform1f(loc_ks, ks)

        loc_ns = glGetUniformLocation(program, "ns")
        glUniform1f(loc_ns, ns)

    # Enviar posição da luz para o shader
    if light_id == 3:  # Para luz 3, usar posição compartilhada com o vagalume
        loc_light_pos = glGetUniformLocation(program, "lightPos3")
        glUniform3f(loc_light_pos, vagalume_luz_position.x, vagalume_luz_position.y, vagalume_luz_position.z)
    elif light_id == 2:
        loc_light_pos = glGetUniformLocation(program, "lightPos2")
        glUniform3f(loc_light_pos, t_x, t_y, t_z)
    else:
        loc_light_pos = glGetUniformLocation(program, "lightPos")
        glUniform3f(loc_light_pos, t_x, t_y, t_z)

    # Matriz de transformação (exemplo de luz representada como um pequeno cubo ou esfera)
    angle = 0.0
    r_x, r_y, r_z = 0.0, 0.0, 1.0
    s_x, s_y, s_z = 1 * esc, 1 * esc, 1 * esc  # Escala pequena para representar a luz
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)

    # Desenho da luz (indicador visual)
    glBindTexture(GL_TEXTURE_2D, 29)
    glDrawArrays(GL_TRIANGLES, 8542980, 8543016 - 8542980)  # Desenha o objeto para representar a luz

In [667]:
def desenha_fan():
    global ka2, kd2, ks2

    # aplica a matriz model
    angle = 0.0
    
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_x =  1.69693 ; t_y = -0.45; t_z = -4.3325;
    
    # escala
    s_x = 0.3; s_y = 0.30; s_z = 0.30;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    loc_ka = glGetUniformLocation(program, "ka2") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka2) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd2") # recuperando localizacao da variavel kd na GPU
    glUniform1f(loc_kd, kd2) ### envia kd pra gpu    
    

    if especular:
        ns = ns_inc # expoente de reflexao especular
        
        loc_ks = glGetUniformLocation(program, "ks2") # recuperando localizacao da variavel ks na GPU
        glUniform1f(loc_ks, ks2) ### envia ks pra gpu        
    
        loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
        glUniform1f(loc_ns, ns) ### envia ns pra gpu        

    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 31)
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES,8543016 ,8570262-8543016) ## renderizando

In [668]:
def desenha_tv():
    global ka1, kd1, ks1

    # aplica a matriz model
    angle = 50.0
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    # translacao
    t_x = 0.31474101543426514 ; t_y = -3.807199478149414; t_z = -2.7757205963134766;
    # escala
    s_x = 0.2; s_y = 0.2; s_z = 0.2;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       

    loc_ka = glGetUniformLocation(program, "ka1") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka1) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd1") # recuperando localizacao da variavel kd na GPU
    glUniform1f(loc_kd, kd1) ### envia kd pra gpu    
    

    if especular:
        ns = ns_inc # expoente de reflexao especular
        
        loc_ks = glGetUniformLocation(program, "ks1") # recuperando localizacao da variavel ks na GPU
        glUniform1f(loc_ks, ks1) ### envia ks pra gpu        
    
        loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
        glUniform1f(loc_ns, ns) ### envia ns pra gpu        

    glBindTexture(GL_TEXTURE_2D, 4)
    glDrawArrays(GL_TRIANGLES, 8570262, 8596134 - 8570262) ## renderizando    

    glBindTexture(GL_TEXTURE_2D, 30)
    glDrawArrays(GL_TRIANGLES, 8596134, 8607234 - 8596134) ## renderizando

### Eventos para modificar a posição da câmera.

* teclas A, S, D e W para movimentação no espaço tridimensional
* Posição do mouse para "direcionar" a câmera
* Pressionar a tecla P alterna entre os modos poligonal e preenchido.
* As teclas Z e X ajustam a escala da borboleta .
* As teclas C e V ajustam a posição do objeto escada.
* As teclas B e N controlam a rotação e a posição da tampa da caixa da pizza.
* A tecla Y imprime a posição atual da câmera no console
* As teclas 1,2,3 são interruptores para as luzes (1-luz da tv, 2-luz do ventilador, 3-vagalume)
* As teclas F e G ajustam luz ambiente
* As teclas H e J ajustam reflexão difusa
* As teclas K e L ajustam reflexão especular
* Translação do vagalume: setas para cima e para baixo (eixo y), para esquerda e direita (eixo x), e teclas 8 e 9 (eixo z)
* ---- extras-----
* Q printa posição vagalume
* U liga/desliga luz ambiente
* I liga/desliga reflexão difusa
* O liga/desliga reflexão especular
* As teclas 5 e 6 ajustam quão pontual ou difuso o reflexo especular é (via ns)

In [670]:
cameraPos   = glm.vec3(1.87914, 2.43606, -2.51148);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);
vagalume_luz_position = glm.vec3(-0.951099, 2.85341, -3.91821)
polygonal_mode = False
cull = False
inc_fov = 0
inc_near = 0
inc_far = 0
inc_view_up = 0
light1_on = True
light2_on = True
light3_on = True
def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode, inc_fov, inc_near, inc_far, cameraUp, inc_view_up
    global pos_, esc_, angrot_, angrot_tampa, pos_tampa, pos_escada2, esc_borboleta, pos_caixa
    global ns_inc, ka1, ks1, kd1, ka2, ks2, kd2, ka3, ks3, kd3, light1_on, light2_on, light3_on
    cameraSpeed = 0.2
    caixaSpeed = 0.2
    
    #eventos vagalume com luz
    if key == 265 and (action == 1 or action == 2):  # seta para cima
        vagalume_luz_position.y += 0.1
    if key == 264 and (action == 1 or action == 2):  # seta para baixo
        vagalume_luz_position.y -= 0.1
    if key == 263 and (action == 1 or action == 2):  # seta para esquerda
        vagalume_luz_position.x -= 0.1
    if key == 262 and (action == 1 or action == 2):  # seta para direita
        vagalume_luz_position.x += 0.1
    if key == 56 and (action == 1 or action == 2):  # tecla 8
        vagalume_luz_position.z -= 0.1
    if key == 57 and (action == 1 or action == 2):  # tecla 9
        vagalume_luz_position.z += 0.1
    if key == 81 and (action == 1 or action == 2):  # tecla Q
        print(vagalume_luz_position.x, vagalume_luz_position.y, vagalume_luz_position.z)
         
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
        if cameraPos[0] >= 5:
            cameraPos[0]=5
        if cameraPos[0] <= -2:
            cameraPos[0]=-2
            
        if cameraPos[1] <= -4:
            cameraPos[1]=-4
        if cameraPos[1] >= 6:
            cameraPos[1]=6

        if cameraPos[2] >= -0.74:
            cameraPos[2]= -0.74
        if cameraPos[2] <= -7.44:
            cameraPos[2]= -7.44

    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
        if cameraPos[0] >= 6.98:
            cameraPos[0]=6.98
        if cameraPos[0] <= -4.37:
            cameraPos[0]=-4.37
        
        if cameraPos[1] <= -4:
            cameraPos[1]=-4
        if cameraPos[1] >= 4.1:
            cameraPos[1]=4.1

        if cameraPos[2] >= -0.74:
            cameraPos[2]= -0.74
        if cameraPos[2] <= -7.44:
            cameraPos[2]= -7.44

    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        if cameraPos[0] >= 6.98:
            cameraPos[0]=6.98
        if cameraPos[0] <= -4.37:
            cameraPos[0]=-4.37
            
        if cameraPos[1] <= -4:
            cameraPos[1]=-4
        if cameraPos[1] >= 4.1:
            cameraPos[1]=4.1

        if cameraPos[2] >= -0.74:
            cameraPos[2]= -0.74
        if cameraPos[2] <= -7.44:
            cameraPos[2]= -7.44
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        if cameraPos[0] >= 6.98:
            cameraPos[0]=6.98
        if cameraPos[0] <= -4.37:
            cameraPos[0]=-4.37
        
        if cameraPos[1] <= -6:
            cameraPos[1]=-6
        if cameraPos[1] >= 4.1:
            cameraPos[1]=4.1

        if cameraPos[2] >= -0.74:
            cameraPos[2]= -0.74
        if cameraPos[2] <= -7.44:
            cameraPos[2]= -7.44
        
    if key == 80 and action==1 and polygonal_mode==True: #tecla P
        polygonal_mode=False
    else:
        if key == 80 and action==1 and polygonal_mode==False:
            polygonal_mode=True
            
    #eventos dos objetos
    if key == 90: #tecla Z
        esc_borboleta = 0.0005
    if key == 88: #tecla X
        esc_borboleta = 0.0003

    if key == 67: #tecla C
        if pos_escada2[1] >= -0.1:
            pos_escada2[1] = -0.1
        pos_escada2[1] += 0.1
    if key == 86: #tecla V
        if pos_escada2[1] <= -2:
            pos_escada2[1] = -2
        pos_escada2[1] -= 0.1
        
    if key == 66: #tecla B
        if angrot_tampa>=199.4:
            angrot_tampa = 199.4
            pos_tampa[1] = -2.89
        angrot_tampa += 1
        pos_tampa[1] += 0.004
        
    if key == 78: #tecla N
        if angrot_tampa <= 144.4:
            angrot_tampa = 144.4
            pos_tampa[1] = -3.1
        angrot_tampa -= 1
        pos_tampa[1] -= 0.004

    if key == 89: #tecla Y
        print(cameraPos)

    ####
     #eventos da luz 
    if key == 85 and (action==1 or action==2): # tecla U - desliga/liga iluminacao ambiente 
        if ka1 == 0 and ka2 == 0 and ka3 == 0:
            ka1, ka2, ka3 = 0.1, 0.1, 0.1
        else:
            ka1, ka2, ka3 = 0, 0, 0

    if key == 73 and (action==1 or action==2): # tecla I - desliga/liga reflexao difusa
        if kd1 == 0 and kd2 == 0 and kd3 == 0:
            kd1, kd2, kd3 = 0.5, 0.5, 0.5
        else:
            kd1, kd2, kd3 = 0, 0, 0

    if key == 79 and (action==1 or action==2): # tecla O - desliga/liga reflexao especular
        if ks1 == 0 and ks2 == 0 and ks3 == 0:
            ks1, ks2, ks3 = 0.9, 0.9, 0.9
        else:
            ks1, ks2, ks3 = 0, 0, 0
        #ajusta quão pontual ou difuso o reflexo especular é (via ns).
        #A mudança em ns afeta a forma e o tamanho do reflexo especular(distribuição da luz refletida)
    if key == 53 and (action==1 or action==2): # tecla 5
            ns_inc = ns_inc * 2
                
    if key == 54 and (action==1 or action==2): # tecla 6
            ns_inc = ns_inc / 2
    #####
    def update_all_lights(parameter, change):
        new_value = max(0.0, min(1.0, parameter + change))  # Limite entre 0 e 1
        return new_value
        
    #Interruptores das luzes 1,2 e 3
    if key == 49 and action == 1:  # tecla 1
        if light1_on:
            ka1, kd1, ks1 = 0, 0, 0
        else: 
            ka1, kd1, ks1 = 0.3, 0.5, 0.9
        light1_on = not light1_on  # Alterna o estado da luz
        
    if key == 50 and action == 1:  # tecla 2
        if light2_on:
            ka2, kd2, ks2 = 0, 0, 0
        else: 
            ka2, kd2, ks2 = 0.3, 0.5, 0.9
        light2_on = not light2_on 
        
    if key == 51 and action == 1:  # tecla 3
        if light3_on:
            ka3, kd3, ks3 = 0, 0, 0
        else:
            ka3, kd3, ks3 = 0.3, 0.5, 0.9
        light3_on = not light3_on

    # Teclas para ajustar iluminação ambiente
    if key == 70 and action in (1, 2):  # tecla F para aumentar ka
        ka1 = update_all_lights(ka1, 0.05)
        ka2 = update_all_lights(ka2, 0.05)
        ka3 = update_all_lights(ka3, 0.05)
    if key == 71 and action in (1, 2):  # tecla G para diminuir ka
        ka1 = update_all_lights(ka1, -0.05)
        ka2 = update_all_lights(ka2, -0.05)
        ka3 = update_all_lights(ka3, -0.05)
    # Teclas para ajustar reflexão difusa
    if key == 72 and action in (1, 2):  # tecla H para aumentar kd
        kd1 = update_all_lights(kd1, 0.05)
        kd2 = update_all_lights(kd2, 0.05)
        kd3 = update_all_lights(kd3, 0.05)
    if key == 74 and action in (1, 2):  # tecla J para diminuir kd
        kd1 = update_all_lights(kd1, -0.05)
        kd2 = update_all_lights(kd2, -0.05)
        kd3 = update_all_lights(kd3, -0.05)
    # Teclas para ajustar reflexão especular
    # ajusta quão brilhante ou opaco o reflexo especular é (via ks)
    # mudança em ks afeta a força do reflexo especular (quantidade de luz refletida).
    if especular:
        if key == 75 and action in (1, 2):  # tecla K para aumentar ks
            ks1 = update_all_lights(ks1, 0.05)
            ks2 = update_all_lights(ks2, 0.05)
            ks3 = update_all_lights(ks3, 0.05)
        if key == 76 and action in (1, 2):  # tecla L para diminuir ks
            ks1 = update_all_lights(ks1, -0.05)
            ks2 = update_all_lights(ks2, -0.05)
            ks3 = update_all_lights(ks3, -0.05)

firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)

    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)

### Matrizes Model, View e Projection

Teremos uma aula específica para entender o seu funcionamento.

In [672]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    
    angle = math.radians(angle)
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade
    
    
    # aplicando translacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
    
    # aplicando rotacao
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    
    # aplicando escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform).T # pegando a transposta da matriz (glm trabalha com ela invertida)_x
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura, inc_fov, inc_near, inc_far
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Nesse momento, nós exibimos a janela!


In [674]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [676]:
pos_poste1   =   [-1.4 , +0.0 , -5.5]
esc_poste    =   8.0
pos_poste2   =   [+4.1 , +0.0 , -2.7]

pos_turtle = [+1.1, -4.8, -3.1]
esc_turtle = 0.013
angrot_turtle = 66

pos_turtle2 = [-0.1, -4.8, -4.3]
angrot_turtle2 = 148.8

pos_turtle3 = [+0.7, -4.8, -3.9]
angrot_turtle3 = 114

pos_turtle4 = [+3.1, -4.8, -3.5]
angrot_turtle4 = 136.8

pos_escada2 =[+0.9, -2, -1.95]
angrot_escada2 = -30

pos_escada =[+0.9, -0.1, -1.95]
esc_escada = 1.2
angrot_escada = -30

pos_borboleta =[-1.5, 2.5, -5.65]
esc_borboleta = 0.0005
angrot_borboleta = -87.5

angrot_tampa = 144.4
pos_tampa =[-0.30000001192092896, -3.1000006198883057, -2.799999952316284]

In [677]:
glEnable(GL_DEPTH_TEST) ### importante para 3D
glLightModelfv ( GL_LIGHT_MODEL_TWO_SIDE , GL_FALSE ) 
ns_inc = 32
ang = 0.1
while not glfw.window_should_close(window):

    glfw.poll_events() 
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)

    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)

    if cull == True:
        glEnable(GL_CULL_FACE) 
        #glCullFace(GL_FRONT)
        glCullFace(GL_BACK)
    else:
        glDisable(GL_CULL_FACE) 

    loc_light_color1 = glGetUniformLocation(program, "lightColor")
    glUniform3f(loc_light_color1, *light_color1)

    loc_light_color2 = glGetUniformLocation(program, "lightColor2")
    glUniform3f(loc_light_color2, *light_color2)

    loc_light_color3 = glGetUniformLocation(program, "lightColor3")
    glUniform3f(loc_light_color3, *light_color3)
    
     # Atualizar posição da luz 
    loc_light_pos1 = glGetUniformLocation(program, "lightPos")
    glUniform3f(loc_light_pos1,0.2489010989665985, -3.9465880393981934, -3.118210792541504)

    loc_light_pos2 = glGetUniformLocation(program, "lightPos2")
    glUniform3f(loc_light_pos2, 1.69, -0.44658949971199036, -4.318209648132324)

    loc_light_pos3 = glGetUniformLocation(program, "lightPos3")
    glUniform3f(loc_light_pos3, vagalume_luz_position.x, vagalume_luz_position.y, vagalume_luz_position.z)

    #Objetos internos
    loc_is_internal = glGetUniformLocation(program, "isInternalObject")
    glUniform1i(loc_is_internal, 1)  # Define como interno
    desenha_turtle(pos_turtle, esc_turtle, angrot_turtle)
    desenha_turtle(pos_turtle2, esc_turtle, angrot_turtle2)
    desenha_turtle(pos_turtle3, esc_turtle, angrot_turtle3)
    desenha_turtle(pos_turtle4, esc_turtle, angrot_turtle4)
    desenha_casa()
    desenha_tapete()
    desenha_rato()
    desenha_pizza()
    desenha_tampa(pos_tampa, angrot_tampa)
    desenha_escada(pos_escada, esc_escada, angrot_escada)
    desenha_escada(pos_escada2, esc_escada, angrot_escada2)
    # desenha_luz(0.1, -0.197497,-4.5,-2.7, 1) #cubo luz da tv
    # desenha_luz(0.05, 1.69693, -0.30, -4.33253, 2) #cubo luz teto
    desenha_fan()
    desenha_tv()
    #Objetos externos
    glUniform1i(loc_is_internal, 0)
    desenha_rua()
    desenha_predios12()
    desenha_predios34()
    desenha_calcada()
    desenha_poste(pos_poste1, esc_poste)
    desenha_poste(pos_poste2, esc_poste)
    desenha_ceu()
    desenha_calcada()
    desenha_borboleta(pos_borboleta, esc_borboleta, angrot_borboleta)
    desenha_bueiro()
    desenha_vagalume()   
    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)    

    if especular:
        # atualizando a posicao da camera/observador na GPU para calculo da reflexao especular
        loc_view_pos = glGetUniformLocation(program, "viewPos") # recuperando localizacao da variavel viewPos na GPU
        glUniform3f(loc_view_pos, cameraPos[0], cameraPos[1], cameraPos[2]) ### posicao da camera/observador (x,y,z)
    
    glfw.swap_buffers(window)

glfw.terminate()